## Langfuse LLM Observability and Prompt Management Tutorial

### What is Langfuse?
Langfuse is a powerful platform for:

* LLM Observability: Track and monitor LLM interactions in production
* Tracing: Detailed execution traces for debugging and optimization
* Analytics: Performance metrics and cost tracking
* Evaluation: Automated quality assessment of LLM outputs

In [1]:
%pip install langfuse langchain langchain-google-genai -q

### Environment Configuration

In [25]:
import os
from google.colab import userdata

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

# Securely get Google API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Purpose: Set up authentication for both Langfuse and ChatGoogleGenerativeAI services.
⚠️ Security Note: In production, use environment variables or secure credential management instead of hardcoding API keys

### Initialize Langfuse Client

In [3]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler

# Initialize Langfuse client (prompt management)
langfuse = get_client()

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_callback_handler = CallbackHandler()

### Prompt Management - Create Prompt Template

In [12]:
langfuse.create_prompt(
    name="event-planner",
    prompt=
    "Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. "
    "The event will be held in {{Location}} on {{Date}}. "
    "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
    "Provide a detailed plan including potential vendors and logistics.",
    config={
        "model":"gemini-2.0-flash",
        "temperature": 0,
    },
    labels=["production"]
);

### Retrieve and Use Prompt

In [13]:
# Get current production version of prompt
langfuse_prompt = langfuse.get_prompt("event-planner")

In [14]:
print(langfuse_prompt.prompt)

Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. The event will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


In [15]:
from langchain_core.prompts import ChatPromptTemplate

langchain_prompt = ChatPromptTemplate.from_template(
        langfuse_prompt.get_langchain_prompt(),
        metadata={"langfuse_prompt": langfuse_prompt},
    )

### Model Configuration from Prompt

In [16]:
model = langfuse_prompt.config["model"]
temperature = str(langfuse_prompt.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gemini-2.0-flash
Temperature: 0


In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7
)

chain = langchain_prompt | model

### Execute with Tracing

In [27]:
example_input = {
    "Event Name": "Wedding",
    "Event Description": "The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.",
    "Location": "Central Park, New York City",
    "Date": "June 5, 2024"
}

In [28]:
# we pass the callback handler to the chain to trace the run in Langfuse
response = chain.invoke(input=example_input,config={"callbacks":[langfuse_callback_handler]})

print(response.content)

Okay, let's plan your event titled **{Event Name}** on June 5, 2024, in Central Park, NYC, based on the description you provide. I'll need you to fill in the blanks below so I can tailor the plan specifically to your needs:

*   **{Event Name}:** (e.g., "Central Park Wellness Fair," "Shakespeare in the Park Picnic," "NYC Community Art Day")
*   **{Event Description}:** (e.g., "A day dedicated to promoting health and wellness resources in the NYC community," "A relaxed picnic and informal performance of Shakespeare's 'A Midsummer Night's Dream'," "A free event showcasing local artists with interactive workshops and live music.")

For this example, let's assume:

*   **{Event Name}:** "Central Park Wellness Fair"
*   **{Event Description}:** "A day dedicated to promoting health and wellness resources in the NYC community, featuring free health screenings, fitness demonstrations, healthy food vendors, and educational workshops."

Here's the detailed event plan:

**Event Plan: Central Park